In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os

In [2]:
input_path = '/Volumes/Behaviour/S1 2P Recordings for Suite2P'
output_path = '/Volumes/Behaviour/Tailored 90sec 2P Trials'
beh_stim = pd.read_csv('/Volumes/Behaviour/stimframes.csv', index_col=0)



In [17]:
directories = [x[0] for x in os.walk(input_path) if os.path.basename(x[0]) == 'plane0']

for directory in directories:
    print('Processing:', directory)
    stop = False
    meta = directory.split('/')  #directory.split('\\') 
    session_name = meta[4] #meta[3] 
    print(session_name)
    
    # load Suite2P outputs
    F = np.load(os.path.join(directory, 'F.npy'))
    Fneu = np.load(os.path.join(directory, 'Fneu.npy'))
    iscell = np.load(os.path.join(directory,'iscell.npy'), allow_pickle=True)
                             
    # read trial timings
    sf_path = meta[0]+'/'+meta[1]+'/'+meta[2]+'/'+meta[3] +'/'+ meta[4]
    sf = pd.read_csv(os.path.join(sf_path, '2P_stimframes.csv'))
    sf = sf.drop(sf.index[sf['stim_qc'] == 'removed']) # discard 2p timings for trials where behaviour videos were deleted because of qc
    cols = list(sf.columns)
    temp_stim = beh_stim[beh_stim['identifier'].str.contains(str(cols[0]))] # subset stimframes to the session being processed
    
    stimuli = list(sf['stimulus'])
    
    for s, stimulus in enumerate(stimuli):
        temp2_stim = temp_stim[temp_stim['identifier'].str.contains(str(stimulus))] # subset to one stimulus
        values = list(temp2_stim['sframe']) # get list of stimframes for one stimulus
        ids = list(temp2_stim['identifier'])
        sf.loc[sf['stimulus']==stimulus,'sframe']  = values
        sf.loc[sf['stimulus']==stimulus, 'identifier'] = ids
        
    sf_qc = sf.drop(sf.index[sf['sframe'] == 'DO NOT USE']) # keep only trials that passed behavioural qc
    sf_qc = sf_qc[~sf_qc[session_name].isnull()] # remove trial without 2P recording
    sf_qc['beh_stim_ms'] = (sf_qc['sframe'].astype(int)*2.5)+2.5 # convert behaviour stimulation frame to msec and add offset (1 frame in ms)
    sf_qc['2p_realstim_frame'] = sf_qc[session_name] + ((sf_qc['beh_stim_ms'] - 163.9) / 163.9)
    sf_qc['2p_realstim_frame'] = sf_qc['2p_realstim_frame'].apply(np.ceil) # round up to next completed frame
    
    sf_qc.to_csv(os.path.join(sf_path, '2P_SF_QC.csv'))
    
    stimframes = np.array(sf_qc['2p_realstim_frame'])
    idx = list(sf_qc['identifier'])
    
    # Subtract neuropil signal from raw traces
    Fc = F - 0.7 * Fneu
    print('matrix shape: ', F.shape)
    
    # Get list of ROIS that are not cells
    iscell = pd.DataFrame(iscell)
    iscell_idx = iscell[0] == 1
    
    # Remove non-cell ROIs from raw traces & raw traces without neuropil
    df_F = pd.DataFrame(F)
    F_isCell = df_F.loc[iscell_idx]
    
    df_Fc = pd.DataFrame(Fc)
    Fc_isCell = df_Fc.loc[iscell_idx]
                             
    # remove ROIs with zero values from raw traces and get new list of indecis corresponding to active ROIs
    F_nonZero = F_isCell.loc[~(F_isCell==0).all(axis=1)]
    activeROIs = list(F_nonZero.index.values)
    print('number of active rois: ', len(activeROIs))

    # keep only active ROIs in F-neuropil
    Fc_Act = df_Fc.loc[df_Fc.index[activeROIs]]
    print('active rois matrix shape: ', Fc_Act.shape)
                             
    if stimframes[0]-183 <0: # determine appropriate length of pre-stimulus baseline based on the number 
        TBL_len = stimframes[0]-1 # of frames available before the very first stimulation
    else:
        TBL_len = 183 # default baseline is 30 s (30 * 6.1 fps = 183)
        
# for each active ROI, calculate 5th percentile, then cut out 30 sec pre-, and 60 sec post-stimulation 
# traces and use these to calculate F0 (I call it T0 since it is trial based) as the mean of Q5 in the trial window.
# Then, compute trial-wise dF_F0 (I called them dT_T0) an crop those to a 3 second window centered sround stimulation.

    dF_F0_short, dF_F0_long = pd.DataFrame(), pd.DataFrame()
    for r, roi in enumerate(activeROIs):
       
        Fc_Q05 = Fc_Act.iloc[r,:].rolling(366, center = True).quantile(0.05) # 5th percentile of neuropil-corrected F trace
        
        tailored_short,  tailored_long = pd.DataFrame(), pd.DataFrame()
            
        for s, stimframe in stimframes:
            start = int(s) - int(TBL_len)
            end = int(s) + 366 # 60 sec * 6.1 fps
            T = Fc_Act.iloc[r,start:end] # trial window of neuropil-corrected F trace
            T_Q05 = Fc_Q05.iloc[start:end] # trial window of 5th percentile trace
            T0 = T_Q05.mean()
            dT_T0_long = (T - T0)/T0
            tailored_long = pd.concat([tailored_long, dT_T0_long], axis=0)
            
            start_short = int(s) - 9 
            end_short = int(s) + 9
            dT_T0_short = dT_T0_long.loc[start_short:end_short]
            tailored_short = pd.concat([tailored_short, dT_T0_short], axis=0)
            
        new_name = session_name + '_' + str(roi)
        dF_F0_long[new_name] = tailored_long[0]
        dF_F0_short[new_name] = tailored_short[0]
    
    print('last short trial length: ', dT_T0_short.shape) # checking length of last trial in frames
    print('last long trial length: ', dT_T0_long.shape)
    print('session baseline length: ', TBL_len)
    
    # add identifier column to dF_F0_short dataframes 
    ids = []
    for i in idx:
        for n in range(19):
            ids.append(i)                                       
    ids = np.array(ids) 
    
    dF_F0_short['identifier'] = ids
 
    dF_F0_short.to_csv(os.path.join(output_path,'dF_F0_SHORT_' +session_name+'.csv'))

#     # add identifier column to dF_F0_long dataframes
#     ids = []
#     if len(dT_T0_long) == TBL_len+366:
#         for i in idx:
#             for n in range(len(dT_T0_long)):
#                 ids.append(i)                                       
#         ids = np.array(ids) 
    
#     else:
#         for i in idx[:-1]:
#             for n in range(int(TBL_len)+366):
#                 ids.append(i) 
#         for n in range(len(dT_T0_long)):
#             ids.append(idx[-1])
#         ids = np.array(ids)
        
#     dF_F0_long['identifier'] = ids
 
#     dF_F0_long.to_csv(os.path.join(output_path,'dF_F0_LONG_' +session_name+'.csv'))

Processing: /Volumes/Behaviour/S1 2P Recordings for Suite2P/2P06_Ses01/suite2p/plane0
2P06_Ses01
matrix shape:  (1040, 17959)
number of active rois:  358
active rois matrix shape:  (358, 17959)


/Users/aspra/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:97: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
/Users/aspra/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:98: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`


last short trial length:  (19,)
last long trial length:  (549,)
session baseline length:  183


/Users/aspra/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:131: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`


Processing: /Volumes/Behaviour/S1 2P Recordings for Suite2P/2P06_Ses02/suite2p/plane0
2P06_Ses02
matrix shape:  (1191, 18865)
number of active rois:  373
active rois matrix shape:  (373, 18865)
last short trial length:  (19,)
last long trial length:  (549,)
session baseline length:  183
Processing: /Volumes/Behaviour/S1 2P Recordings for Suite2P/2P06_Ses03/suite2p/plane0
2P06_Ses03
matrix shape:  (1101, 19950)
number of active rois:  283
active rois matrix shape:  (283, 19950)
last short trial length:  (19,)
last long trial length:  (354,)
session baseline length:  183
Processing: /Volumes/Behaviour/S1 2P Recordings for Suite2P/2P05_Ses04/suite2p/plane0
2P05_Ses04
matrix shape:  (1106, 18259)
number of active rois:  330
active rois matrix shape:  (330, 18259)
last short trial length:  (19,)
last long trial length:  (482,)
session baseline length:  183
Processing: /Volumes/Behaviour/S1 2P Recordings for Suite2P/2P06_Ses04/suite2p/plane0
2P06_Ses04
matrix shape:  (1362, 19757)
number of 

KeyboardInterrupt: 

In [12]:
idx[-1]

'2P06_Ses03_29OCT2021_opto_Trial_3_87161'

In [13]:
plt.figure(figsize=(10, 3))
#plt.plot(dF_F0.iloc[:,200], alpha=0.5)
plt.plot(dF_F0, alpha=0.5)
plt.ylim([-0.05,0.2])
plt.xlim([350,400])
plt.eventplot(stimframes[0:1], color = 'fuchsia', linelength = 800)
plt.show()

NameError: name 'dF_F0' is not defined

<Figure size 720x216 with 0 Axes>

In [59]:
new_sf = pd.read_csv('/Volumes/Behaviour/S1 2P Recordings for Suite2P/2P05_Ses07/2P_SF_QC.csv', index_col=0)

In [62]:
stimframes = np.array(new_sf['2p_realstim_frame'])
stimframes

array([  366.,   870.,  2330.,  3094.,  4520.,  5264.,  6052.,  6770.,
        7488.,  8238.,  9690., 10426., 11343., 12017., 12977., 13705.,
       14506., 15139., 15949., 16696., 17416., 18111.])

In [56]:
dF_F0.tail()

,2P05_Ses07_0,2P05_Ses07_1,2P05_Ses07_2,2P05_Ses07_4,2P05_Ses07_5,2P05_Ses07_6,2P05_Ses07_7,2P05_Ses07_9,2P05_Ses07_10,2P05_Ses07_11,...,2P05_Ses07_842,2P05_Ses07_845,2P05_Ses07_847,2P05_Ses07_849,2P05_Ses07_850,2P05_Ses07_857,2P05_Ses07_858,2P05_Ses07_859,2P05_Ses07_873,2P05_Ses07_875
18116,0.007274,0.009594,0.016154,0.012755,0.012601,0.005522,0.022797,0.000782,0.017764,0.002138,...,0.014274,0.006585,0.002064,0.010006,0.004119,0.004888,0.012644,0.010506,0.006797,0.005026
18117,0.000848,0.020033,0.012975,0.004948,0.006564,0.007601,0.009279,0.004967,0.012463,0.008305,...,0.010004,0.014591,0.002722,0.014247,0.008712,0.010368,0.006218,0.014545,0.009126,0.005018
18118,0.015873,0.017331,0.000572,0.010645,0.010327,0.002787,0.020085,0.007938,0.016114,0.008220,...,0.011768,0.003997,0.002438,0.006437,0.007320,0.007625,0.008716,0.001237,0.005444,-0.001451
18119,0.006125,0.023420,0.007341,0.004584,0.008093,0.006218,0.004116,0.007280,0.003837,-0.000198,...,0.012369,0.003872,0.001193,0.013357,0.010058,0.014140,0.002168,0.005111,0.008571,0.004843
18120,0.003509,0.012961,0.010367,0.008149,0.008966,-0.000423,0.005436,0.006023,0.014232,0.007811,...,0.017168,0.010441,0.003496,0.008729,0.001196,0.002658,0.007830,0.014224,0.017671,0.011982
